# Beispiel 2.11
Wir betrachten erneut das lineare Optimierungsproblem aus Beispiel 1.2 aus der Vorlesung.
Das ist

$
\begin{align*}
\min \; -3x_1-5x_2 \quad \text{ u.d.N. } \quad 3x_1+2x_2 &\leq 18\\
x_1 &\leq 4\\
2x_2 &\leq 12\\
x &\geq 0_2.
\end{align*}
$

Dieses Problem überführen wir dann in Normalform. Wir erhalten damit

$
\begin{align*}
\min \; c^\top x \quad \text{ u.d.N. } Ax &= b\\
x &\geq 0_5.
\end{align*}
$

mit $c = \begin{pmatrix}-3 \\ -5 \\ 0 \\ 0 \\ 0 \end{pmatrix}, \quad A = \begin{pmatrix} 3&2&1&0&0\\1&0&0&1&0\\0&2&0&0&1 \end{pmatrix}, \quad b = \begin{pmatrix} 18\\4\\12 \end{pmatrix}.$

In diesem Beispiel werden wir uns nun den Ecken für dieses LP zuwenden.
Zunächst importieren wir die benötigten Pakete:

In [ ]:
from scipy.optimize import linprog
import numpy as np
import matplotlib.pyplot as plot

Anschließend geben wir dann unsere Parameter an.
Wir werden für dieses Beispiel durchgehend Numpy-Arrays verwenden:

In [ ]:
c = np.array([-3,-5,0,0,0])
A = np.block([np.array([[3,2],[1,0],[0,2]]),np.eye(3)]) #auch np.hstack hätte es hier getan
b = np.array([18,4,12])

Nun lösen wir zunächst das LP und lassen uns das Ergebnis ausgeben:

In [ ]:
res = linprog(c, A_eq=A, b_eq=b, bounds=(0,None))
print("Optimallösung: " + str(res.x))
print("Optimalwert: " + str(res.fun))

Wir erhalten den gleichen Optimalwert wie auch schon in Beispiel 1.2 und das ist ein gutes Indiz dafür, dass wir bis hierhin keine Fehler gemacht haben.
Als nächstes überprüfen wir kurz den Rang von $A$:

In [ ]:
print("Rang(A) = " + str(np.linalg.matrix_rank(A)))

Wenn Rang(A) = 3 gilt, dann können also Ecken maximal drei von 0 verschiedene Einträge haben.
Überprüfen wir nun also für ein paar Vektoren, ob diese Ecken sind.
Dazu nutzen wir Lemma 2.9 aus der Vorlesung.
Zunächst lagern wir den Eckentest in eine Funktion `istEcke` aus:

In [ ]:
def istEcke(x,Aeq,beq):
    if all(x >= np.zeros(5)) and (np.linalg.norm(Aeq@x-beq) <= 1e-6):
        J = np.ma.nonzero(x)[0]
        if (np.linalg.matrix_rank(A[:,J]) == len(J)):
            # x ist Ecke nach Lemma 2.9 a)
            return 1
        else:
            # x ist zulässig aber keine Ecke
            return 0
    else:
        # x ist unzulässig
        return -1

Es ist an dieser Stelle sicher eine Bemerkung wert, dass `np.zeros(5)` dem Befehl `np.zeros((1,5))` entspricht und nicht etwa `np.zeros((5,5))` - gerade auch im Unterschied zu MATLAB.
Zudem wird die Matrixmultiplikation mi @ durchgeführt.
Der Operator * ist für punktweise Multiplikation zu verwenden.

Nun definieren wir eine kleine Auswahl an Vektoren und überprüfen diese:

In [ ]:
x1 = np.array([2,6,0,2,0])
x2 = np.array([2,0,12,2,12])
x3 = np.array([2,6,0,2,1])
x4 = np.array([2,-6,0,2,0])
x5 = np.array([4,3,0,0,6])

for x in [x1,x2,x3,x4,x5]:
    result = istEcke(x,A,b)
    if result == -1:
        print(str(x) + " ist unzulässig")
    elif result == 0:
        print(str(x) + " ist zulässig aber keine Ecke")
    else:
        print(str(x) + " ist zulässig und Ecke")

Jetzt wollen wir uns das noch grafisch darstellen.
Wir werden als Vorlage die Darstellung der Höhenlinien aus Beispiel 1.2 verwenden und dann noch die zulässige Menge dort einzeichnen:

In [ ]:
# Der Plot der Höhenlinien aus Beispiel 1.2
X1,X2 = np.meshgrid(np.linspace(-1,7,50),np.linspace(-1,7,50));
Z = c[0]*X1 +c[1]*X2;
fig, ax = plot.subplots()
C = ax.contour(X1,X2,Z,range(-54,0,6))

# Die Restriktionsmenge (charakterisiert über Ecken)
Ecken = np.array([[0,4,4,2,0,0],[0,0,3,6,6,0]])
plot.plot(Ecken[0,:],Ecken[1,:],lw=2)

ax.clabel(C, C.levels);
ax.set_title('Höhenlinien für Beispiel 1.2 und Ecken nach Beispiel 2.11')
ax.set_xlabel('x1');
ax.set_ylabel('x2');